# UCSD ECE143 Final Project

## 1. Data Clean Up

In [2]:
import json
from collections import defaultdict

companies = defaultdict(int)
location = defaultdict(int)
skills = defaultdict(int)
industry = defaultdict(int)

with open('143data.json','r') as f:
    allpeople = f.readlines()
    
    for person in allpeople:
        info = json.loads(person)

#         print(info["lastName"], info["firstName"])
        industry[info['industryName']] += 1

        if not info['student']:
            companies[info['experience'][0]['companyName']] += 1
            
        curPlace = info['geoLocationName']
        city, state = curPlace.split(',')
        found = None #indicator of whether location has found in certain format
        for savedCity, savedState in location:
            if city in savedCity:
                found = True
                location[(savedCity, savedState)] += 1
                
        if not found:
            location[(city,state)] += 1
        
        for skill in info['skills']:
            if skill['name'].lower() == 'c' or skill['name'].lower() == 'c++' or skill['name'].lower() == 'c/c++' :
                skills['c/c++'] += 1
            else:
                skills[skill['name'].lower()] += 1
print(companies)
print(location)
print(industry)
print(skills)
        
            

defaultdict(<class 'int'>, {'Amazon': 7, 'Uber': 1})
defaultdict(<class 'int'>, {('San Diego County', ' California'): 8})
defaultdict(<class 'int'>, {'Internet': 1, 'Computer Software': 6, 'Higher Education': 1})
defaultdict(<class 'int'>, {'java': 8, 'javascript': 5, 'spring framework': 1, 'agile': 1, 'spring boot': 1, 'python': 7, 'react': 1, 'sql': 3, 'django': 1, 'mysql': 2, 'cascading style sheets (css)': 1, 'html': 3, 'linux': 5, 'hibernate': 1, 'git': 2, 'mybatis': 1, 'shiro': 1, 'redis': 1, 'data analysis': 1, 'software development': 2, 'front-end development': 1, 'image processing': 1, 'c/c++': 8, 'tensorflow': 1, 'network': 1, 'typescript': 1, 'amazon web services (aws)': 2, 'pytorch': 1, 'latex': 1, 'matlab': 2, 'r': 1, 'ocaml': 2, 'scala': 1, 'hadoop': 1, 'css': 1, 'apache spark': 2, 'elasticsearch': 1, 'distribued systems': 1, 'machine learning': 1, 'spring mvc': 1, 'node.js': 1, 'embedded software': 1, 'leadership': 1, 'big data analytics': 1, 'http': 1, 'mongodb': 1, 'we

## 2. Data Visualization

### a. Employees Skills --- Pie Plot

In [65]:
import geoplotlib
import json
from collections import defaultdict
import plotly
# import plotly.plotly as py
import plotly.offline as of
import plotly.graph_objs as go

# seperate the dict to be two lists
def seperateData(data):
    dictionary = dict(data)
    keys = dictionary.keys()
    values = dictionary.values()
    return list(keys), list(values)

# Set the top most skills we want to display
def getHighest(key, values, num):
    newKey = key[:num]
    newValue = values[:num]
    if num < len(key):
        newKey.append("Others")
        newValue.append(sum(values[num:]))
    return newKey, newValue
    

if __name__ == '__main__':
    
    #=============== Set up the data ===================#
    skill, number = seperateData(skills)
    number = [number[i]/sum(number) for i in range(len(number))] #convert number to percentage
    skill, number = getHighest(skill, number, 15)  # Only display the top NO.15 skills

    #===============Then Set up the figures ============#
    of.offline.init_notebook_mode(connected=True)
    trace = [go.Pie(
        labels = skill, 
        values = number, 
        hole =  0,                          # Modify the radius of the hole, higher means larger hole
        hoverinfo = "label + percent")]
    layout = go.Layout(title ="Skills UCSD alumni mastered", titlefont=dict(color='rgb(200, 100, 100)',size=30))
    data = [trace]
    fig = go.Figure(data = trace, layout = layout)
    of.iplot(fig)


### b. People in different companies  -- Bar Chart

In [51]:
if __name__ == '__main__':
    #=============== Set up the data ===================#
    company, number = seperateData(companies)
    company, number = getHighest(company, number, 15)  # Only display the top NO.15 companies
    
    #============== Set up the Figures =================#
    of.offline.init_notebook_mode(connected=True)
    trace = [go.Bar(
        x = company, 
        y = number, 
        orientation='v'
    )]
    layout = go.Layout(plot_bgcolor='#E6E6FA', title =" Companies UCSD alumni went to", titlefont=dict(color='rgb(200, 100, 100)',size=30))
    data = [trace]
    fig = go.Figure(data = trace, layout = layout)
    of.iplot(fig)
    

### c. Industry that UCSD Alumni went to -- radar chart
#### reference links: https://plot.ly/python/radar-chart/

In [ ]:
if __name__ == '__main__':
    #=============== Set up the data ===================#
    industries, number = seperateData(industry)
    print(industries)
    industries, number = getHighest(industries, number, 15)  # Only display the top NO.15 industries
    
    #============== Set up the Figures =================#
    fig = go.Figure()
    fig.add_trace(go.Scatterpolar(
      r=number,
      theta=industries,
      fill='toself',
      name='Industry that UCSD Alumni went to',
      opacity  = 0.5, # Set up the opacity of the figure
    ))
    
    fig.update_layout(
    polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 8]
    )),
    showlegend=True
        
)
    fig.show()

In [63]:
a = ['abbb','basd','cqwq','dad']
a[:2]

['abbb', 'basd']